In [110]:
import pandas as pd
import numpy as np

# Ustawienia wyświetlania
pd.set_option('display.max_rows', None)       # pokaż wszystkie wiersze
pd.set_option('display.max_columns', None)    # pokaż wszystkie kolumny
pd.set_option('display.width', None)          # brak ograniczenia szerokości
pd.set_option('display.max_colwidth', None)   # pełna szerokość kolumny (np. dla tekstu)


In [111]:
stock_list = pd.read_csv('stock_list.csv')
income = pd.read_csv('income.csv')
market_val = pd.read_csv('market_value.csv')
profitability = pd.read_csv('profitability.csv')
cash_flow = pd.read_csv('cash_flow.csv')
liabilities = pd.read_csv('liabilities.csv')
liquidity = pd.read_csv('liquidity.csv')
activity = pd.read_csv('activity.csv')

In [112]:
# Łączenie DataFrame'ów po kolei bez użycia reduce
all_data = income.merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(cash_flow, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liquidity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(activity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

# Podgląd wyniku
all_data.head()

,Nazwa,Ticker,Kwartały,Data,Przychody ze sprzedaży,Zysk ze sprzedaży,Zysk operacyjny (EBIT),Zysk z działalności gospodarczej,Zysk przed opodatkowaniem,Zysk netto,EBITDA,Kurs,Wartość księgowa na akcję,Cena/WK,Zysk na akcję,Cena/Zysk,Przychody ze sprzedaży na akcję,Cena/Przychód,Zysk operacyjny,Cena/Zysk operacyjny na akcję,Wartość księgowa Grahama na akcję,Cena/Wartość księgowa Grahama,Wartość przedsiębiorstwa na akcję,Wartość przedsiębiorstwa/Przychody,Wartość przedsiębiorstwa/EBITDA,Wartość przedsiębiorstwa/EBIT,ROE,ROA,Marża zysku operacyjnego,Marża zysku netto,Marża zysku ze sprzedaży,Marża zysku brutto,Marża zysku brutto ze sprzedaży,Rentowność operacyjna aktywów,Udział zysku netto w przepływach operacyjnych,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zadłużenie długoterminowe,Zadłużenie środków trwałych,Pokrycie aktywów trwałych kapitałami stałymi,Trwałość struktury finansowania,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,Zadłużenie netto,Zadłużenie netto / EBITDA,Zadłużenie finansowe netto,Zadłużenie finansowe netto / EBITDA,I stopień pokrycia,II stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Płynność podwyższona,Pokrycie zobowiązań należnościami,Udział kapitału pracującego w aktywach,Pokrycie kosztów kapitałem obrotowym,Rotacja należności,Cykl należności,Cykl zobowiązań,Rotacja zapasów,Cykl zapasów,Rotacja majątku obrotowego,Rotacja majątku trwałego,Rotacja majątku ogółem,Cykl operacyjny,Cykl konwersji gotówki
0,06MAGNA,06N,2005/Q4,2006-02-14,-9 702,-283,-283,-2 457,-2 457,-785,NaN,"62,00","69,22","0,90","0,93","66,94","2,59","23,93","-0,30",NaN,"44,87","1,38","32,04","12,37","-118,18","-108,22","1,34%","0,89%","-11,43%","35,75%","-11,43%","-11,43%",NaN,"-0,28%","6,28%","3,70%","0,24","0,36",NaN,NaN,NaN,NaN,"0,35","0,71",-73 599 000,"90,31",-90 034 000,"110,47","2,01","2,01","1,65","2,77","2,82","1,99","0,09","0,43","15,68","0,42",868,2 920,"1,49",245,"0,04","0,07","0,03",1 113,-1 807
1,06MAGNA,06N,2006/Q1,2006-05-15,1 150,-667,-667,-667,-667,633,NaN,"69,00","69,50","0,99","0,86","79,89","-4,08",NaN,"-0,60",NaN,"45,11","1,53","43,16",NaN,"-51,48","-72,08","1,24%","0,85%","14,68%","-21,17%","14,68%","14,68%",NaN,"-0,59%","7,64%","11,65%","0,22","0,32",NaN,NaN,NaN,NaN,"0,33","0,67",-76 836 000,"30,49",-77 652 000,"30,81","2,01","2,01","1,60","2,93","3,00","2,13","0,13","0,44","-13,92","-0,74",-496,-1 975,"-1,76",-207,"-0,06","-0,11","-0,04",-703,1 272
2,06MAGNA,06N,2006/Q2,2006-08-14,2 799,1 539,1 539,1 539,1 539,3 707,NaN,"76,00","70,47","1,08","1,14","66,93","0,47","160,75","-0,45",NaN,"77,74","0,98","29,79","63,01","-39,60","-66,57","1,61%","1,06%","-94,65%","240,18%","-94,65%","-374,52%",NaN,"-0,42%","5,95%",NaN,"0,25","0,38",NaN,NaN,NaN,NaN,"0,25","5,79",-63 315 000,"28,00",-138 888 000,"61,43","22,72","22,72","1,77","3,85","3,93","1,79","0,11","0,72","63,46","0,12",3 118,17 994,"0,48",755,"0,01","0,02","0,00",3 873,-14 121
3,06MAGNA,06N,2006/Q3,2006-11-14,18 282,17 507,17 507,17 507,17 507,20 167,NaN,"88,00","31,50","2,79","8,03","10,96","4,17","21,11","6,02","14,62","46,12","1,91","79,93","19,18","13,31","13,28","25,49%","9,95%","144,43%","192,65%","144,43%","127,08%",NaN,"7,46%","39,45%",NaN,"0,35","0,88","0,05","4,04","4,93","0,41","0,38","1,77",46 372 000,"2,57",-24 249 000,"-1,34","4,71","4,93","0,38","2,71","2,81","0,47","0,10","0,59","-25,94","1,58",230,2 194,"-1,02",-358,"0,05","0,21","0,04",-128,-2 322
4,06MAGNA,06N,2006/Q4,2007-03-01,-16 331,-25 673,-25 695,-25 695,-25 695,2 235,NaN,"97,60","30,73","3,18","7,65","12,77","1,96","49,72","-2,43",NaN,"39,63","2,46","77,62","39,54","-32,04","-31,89","24,88%","12,09%","-124,00%","389,46%","-123,63%","-124,00%",NaN,"-3,85%","83,15%",NaN,"0,37","0,77",NaN,NaN,NaN,NaN,"0,37","1 111,18",7 704 000,"-1,06",-60 059 000,"8,25","2 979,26","2 979,26","0,85","2,68","2,68","0,89","0,00","0,63","9,05","0,95",385,4 607,"2,94",124,"0,03","0,18","0,02",509,-4 098


In [113]:
def clean_numeric_column(col):
    # Zamień wartości na stringi i oczyść z spacji oraz przecinków
    col_cleaned = col.astype(str).str.replace(' ', '', regex=False).str.replace(',', '.', regex=False).str.replace('%', '', regex=False)

    # Ustaw NaN dla pustych lub błędnych stringów
    col_cleaned = col_cleaned.where(~col_cleaned.isin(['', 'nan', 'NaN', 'None']), np.nan)

    # Spróbuj zamienić na liczby
    numeric_col = pd.to_numeric(col_cleaned, errors='coerce')

    # Jeśli większość wartości po konwersji to liczby (np. > 80%), przyjmujemy że kolumna jest liczbowa
    ratio_numeric = numeric_col.notna().sum() / len(numeric_col)
    if ratio_numeric > 0.4:
        return numeric_col  # konwersja udana
    else:
        return col  # zachowaj oryginalną kolumnę (tekstową)

# Przetwórz tylko kolumny typu object lub string
for col in all_data.select_dtypes(include=['object', 'string']).columns:
    all_data[col] = clean_numeric_column(all_data[col])

all_data['Data'] = pd.to_datetime(all_data['Data'], errors='coerce')

In [114]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6157 entries, 0 to 6156
Data columns (total 67 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   Nazwa                                          6157 non-null   object        
 1   Ticker                                         6157 non-null   object        
 2   Kwartały                                       6157 non-null   object        
 3   Data                                           6157 non-null   datetime64[ns]
 4   Przychody ze sprzedaży                         5961 non-null   float64       
 5   Zysk ze sprzedaży                              5961 non-null   float64       
 6   Zysk operacyjny (EBIT)                         5961 non-null   float64       
 7   Zysk z działalności gospodarczej               5961 non-null   float64       
 8   Zysk przed opodatkowaniem                      5961 non-nu

In [115]:
all_data.to_csv('app_all_data.csv', index=False)

In [116]:
indicators = income[['Nazwa', 'Ticker', 'Kwartały', 'Data']].merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

indicators['Data'] = pd.to_datetime(indicators['Data'], errors='coerce')

indicators_latest = indicators.loc[indicators.groupby('Ticker')['Data'].idxmax()].reset_index(drop=True)

for col in indicators_latest.select_dtypes(include=['object', 'string']).columns:
    indicators_latest[col] = clean_numeric_column(indicators_latest[col])

indicators_latest.drop(columns=['Kwartały', 'Data', 'Kurs'], inplace=True)

indicators_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 36 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Nazwa                                         133 non-null    object 
 1   Ticker                                        133 non-null    object 
 2   Wartość księgowa na akcję                     133 non-null    float64
 3   Cena/WK                                       122 non-null    float64
 4   Zysk na akcję                                 133 non-null    float64
 5   Cena/Zysk                                     76 non-null     float64
 6   Przychody ze sprzedaży na akcję               133 non-null    float64
 7   Cena/Przychód                                 124 non-null    float64
 8   Zysk operacyjny                               133 non-null    float64
 9   Cena/Zysk operacyjny na akcję                 79 non-null     flo

In [117]:
indicators_latest.to_csv('app_indicators.csv', index=False)

In [118]:
income_yearly = income[income['Kwartały'].str.contains('Q4', na=False)].copy()


for col in income_yearly.select_dtypes(include=['object', 'string']).columns:
    income_yearly[col] = clean_numeric_column(income_yearly[col])

income_yearly['Data'] = pd.to_datetime(income_yearly['Data'], errors='coerce')

income_yearly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1620 entries, 3 to 10666
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Nazwa                             1620 non-null   object        
 1   Ticker                            1620 non-null   object        
 2   Kwartały                          1620 non-null   object        
 3   Data                              1620 non-null   datetime64[ns]
 4   Przychody ze sprzedaży            1571 non-null   float64       
 5   Zysk ze sprzedaży                 1571 non-null   float64       
 6   Zysk operacyjny (EBIT)            1571 non-null   float64       
 7   Zysk z działalności gospodarczej  1571 non-null   float64       
 8   Zysk przed opodatkowaniem         1571 non-null   float64       
 9   Zysk netto                        1620 non-null   float64       
 10  EBITDA                            0 non-null      fl

In [119]:
income_yearly.to_csv('app_income_yearly.csv', index=False)